# Застосування генетичного алгоритму для вирішення задачі про рюкзак
## Описуємо сутність продукту у класі та створюємо перелік товарів

In [495]:
class Product():
  def __init__(self, name, space, price):
    self.name = name
    self.space = space
    self.price = price

  def __str__(self):
      return f"Name: {self.name}, Space: {self.space}, Price: {self.price}"

In [496]:
products_list = []
products_list.append(Product('Refrigerator A', 0.751, 999.90))
products_list.append(Product('Cell phone', 0.00000899, 2199.12))
products_list.append(Product('TV 55', 0.400, 4346.99))
products_list.append(Product("TV 50' ", 0.290, 3999.90))
products_list.append(Product("TV 42' ", 0.200, 2999.00))
products_list.append(Product("Notebook A", 0.00350, 2499.90))
products_list.append(Product("Ventilator", 0.496, 199.90))
products_list.append(Product("Microwave A", 0.0424, 308.66))
products_list.append(Product("Microwave B", 0.0544, 429.90))
products_list.append(Product("Microwave C", 0.0319, 299.29))
products_list.append(Product("Refrigerator B", 0.635, 849.00))
products_list.append(Product("Refrigerator C", 0.870, 1199.89))
products_list.append(Product("Notebook B", 0.498, 1999.90))
products_list.append(Product("Notebook C", 0.527, 3999.00))

## Визначаємо фітнес функцію для вирішення задачі про рюкзак

In [497]:
def fitness_func(ga_instance, solution, solution_idx):
    capacity = 3
    total_price = 0
    products_set = set()
    if len(set(solution)) != 5:
        return 0
    for product_idx in solution:
        product = products_list[product_idx]
        if product.name in products_set:
            continue
        if product.space <= capacity:
            total_price += product.price
            capacity -= product.space
        else:
            break
    return total_price

## Створюємо утилітарні функції для відображження інформації та графіку навчання

In [498]:
def print_solution(solution, solution_idx):
    idx = int(solution_idx)
    print(f"Solution #{idx}:", end=" ")
    for product_idx in solution:
        print(products_list[int(product_idx)].name, end=" ")
    print(f"Total price: {sum(products_list[product_idx].price for product_idx in solution)}")
    print(f"Total space: {sum(products_list[product_idx].space for product_idx in solution)}")

In [499]:
def get_total_price(solution):
    return sum([products_list[ix].price for ix in solution])

In [500]:
from pygad import pygad
import plotly.express as px

def show_learning_curve(ga: pygad.GA):
    # Беремо найкращу пристосованість за кожне покоління з PyGAD
    history = getattr(ga, "best_solutions_fitness", None)
    if not history:
        raise ValueError("Історія навчання відсутня: ga.best_solutions_fitness порожня або не створена.")
    x = list(range(len(history)))
    fig = px.line(x=x, y=history,
                  title='Зміна загальної ціни товарів у вантажівці за поколіннями')
    fig.update_layout(xaxis_title="Покоління", yaxis_title="Найкраща ціна")
    fig.show()

## Створюємо обʼєкт навчання за генетичним алгоритмом та беремо найкращий розвʼязок задачі рюкзака

In [501]:
from pygad import pygad

ga_instance = pygad.GA(
    num_generations=1000,        # Кількість поколінь
    num_parents_mating=5,       # Скільки особин схрещуються
    fitness_func=fitness_func,  # Наша функція
    sol_per_pop=1000,             # Особин у популяції
    num_genes=5,                # Кількість генів у хромосомі
    gene_type=int,            # Тип генів
    init_range_low=1,
    init_range_high=14,
    mutation_percent_genes=25,  # 25% генів мутують
)

In [502]:
ga_instance.run()
solution, fitness, solution_idx = ga_instance.best_solution()
print_solution(solution, solution_idx)

Solution #0: TV 42'  TV 50'  Notebook A TV 55 Notebook C Total price: 17844.79
Total space: 1.4205


## Виводимо криву навчання

In [503]:
show_learning_curve(ga_instance)

## Висновки

З проведеного дослідження бачимо, що генетичний алгоритм вирішує поставлену задачу досить наближено до оптимального рішення. За моїми спостереженнями він працює точніше, чим більше поколінь ми встановлюємо та при невеликій частці генів, котрим дозволено мутувати. Досягнення поставленої задачі вимагає також чіткого розуміння, як створити фітнес функцію.